In [157]:
import os
import glob

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,DoubleType

spark = SparkSession.builder.appName("PF").config("spark.sql.caseSensitive", "True").getOrCreate()

In [158]:
#read inputs 
acc_meta = spark.read.options(inferSchema='True',header='True').csv('./data/input/account_metadata.csv')


#read master ledger file
path = glob.glob('./data/input/Master Ledger.xlsx - Master_Ledger.csv')
df = spark.read.options(inferSchema='True',header='True').csv(path)
df = df.withColumn("ID",col("ID").cast(IntegerType()))\
        .withColumn("Amount",col("Amount").cast(DoubleType()))\
        .withColumn("Subscriptions",col("Subscriptions").cast(BooleanType()))\
        .withColumn("Return",col("Return").cast(BooleanType()))\
        .withColumn("Real Amount",col("Real Amount").cast(DoubleType()))

df = df.withColumn("Date",to_date(col("Date"),"MM/dd/yyyy"))
df.printSchema()
df = df.dropna(how="all",subset= ["ID"])
df.orderBy("ID", ascending=False).show()

root
 |-- ID: integer (nullable = true)
 |-- Item: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Account: string (nullable = true)
 |-- Account Type: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- Transaction Type: string (nullable = true)
 |-- Categories: string (nullable = true)
 |-- Categories 2: string (nullable = true)
 |-- Real Amount: double (nullable = true)
 |-- Note: string (nullable = true)
 |-- Subscriptions: boolean (nullable = true)
 |-- Return: boolean (nullable = true)

+----+--------------+-------+----------+--------------------+------------+-----+----------------+-------------+------------+-----------+--------------------+-------------+------+
|  ID|          Item| Amount|      Date|             Account|Account Type|Owner|Transaction Type|   Categories|Categories 2|Real Amount|                Note|Subscriptions|Return|
+----+--------------+-------+----------+--------------------+------------+-

In [159]:
#read all csv files exported from Empower
path = glob.glob('./data/*.csv')
df2 = spark.read.options(inferSchema='True',header='True').csv(path)


#add more columns to df2
df2 = df2.withColumn("ID",lit(1))\
.withColumn("Item",col("Description")).drop("Description")\
.withColumn("Real Amount",col("Amount"))\
.withColumn("Amount",abs(col("Amount")))\
.withColumn("Account Type",lit(None))\
.withColumn("Owner",lit(None))\
.withColumn("Transaction Type",when(col("Real Amount") <0, "Expense").otherwise("Income"))\
.withColumn("Categories",lit(None))\
.withColumn("Categories 2",col("Category")).drop("Category")\
.withColumn("Owner",lit(None))\
.withColumn("Subscriptions",lit(False))\
.withColumn("Return",lit(False))\
.drop("Tags")

df2.printSchema()

df2.show()

root
 |-- Date: date (nullable = true)
 |-- Account: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ID: integer (nullable = false)
 |-- Item: string (nullable = true)
 |-- Real Amount: double (nullable = true)
 |-- Account Type: void (nullable = true)
 |-- Owner: void (nullable = true)
 |-- Transaction Type: string (nullable = false)
 |-- Categories: void (nullable = true)
 |-- Categories 2: string (nullable = true)
 |-- Subscriptions: boolean (nullable = false)
 |-- Return: boolean (nullable = false)

+----------+--------------------+------+---+--------------------+-----------+------------+-----+----------------+----------+--------------+-------------+------+
|      Date|             Account|Amount| ID|                Item|Real Amount|Account Type|Owner|Transaction Type|Categories|  Categories 2|Subscriptions|Return|
+----------+--------------------+------+---+--------------------+-----------+------------+-----+----------------+----------+--------------+----------

In [160]:
# union master ledger with empower, then 
min_date = df2.select(min("Date")).first()[0]
df3 = df.filter(col("Date")<min_date).unionByName(df2, allowMissingColumns=True)
df3 = df3.drop("ID","Account Type").join(acc_meta, on = 'Account')

df3.show()

+--------------------+--------------------+-------+----------+-----+----------------+-------------+------------+-----------+--------------------+-------------+------+------------+
|             Account|                Item| Amount|      Date|Owner|Transaction Type|   Categories|Categories 2|Real Amount|                Note|Subscriptions|Return|Account Type|
+--------------------+--------------------+-------+----------+-----+----------------+-------------+------------+-----------+--------------------+-------------+------+------------+
|Checking BoA PA 7462|    Checking Account|2541.59|2021-03-24|   PA|         Income2|        Other|       Other|    2541.59|             Account|        false| false|       Debit|
|BoA Customized Ca...|      Credit Account|1583.95|2021-03-24|   PA|        Expense2|        Other|       Other|   -1583.95|             Account|        false| false|      Credit|
|Checking BoA PA 7462|         Vay Tùng Hà| 3000.0|2021-03-24|   PA|         Income2|       People| 

In [161]:
df3.toPandas().to_csv("./data/output/out.csv", index_label="ID")